In [1]:
import torch
import torch.nn as nn

from tqdm import tqdm
import wandb

import numpy as np
import pandas as pd

from models.transformer import Transformer

In [ ]:
Xtrain = torch.tensor(np.load('../data/Xtrain_base.npy').astype(int))
Ytrain = torch.tensor(np.load('../data/Ytrain_base.npy').astype(int))

Xtest = torch.tensor(np.load('../data/Xtest_base.npy').astype(int))
Ytest = torch.tensor(np.load('../data/Ytest_base.npy').astype(int))

vocab = pd.read_csv('../data/vocab.csv', header=None, index_col=0)
vocab_size = vocab.shape[0]
cat_label_mapping = pd.read_csv('../data/cat_label_mapping.csv')
num_classes = cat_label_mapping.shape[0]
context_length = Xtrain.shape[1]


net = Transformer(num_layers=4, vocab_size=vocab_size, num_tokens=context_length, d_model=16, d_q_k_v=16, num_heads=4, num_classes=num_classes)
optim = torch.optim.AdamW(params = net.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs=100
batch_size=2

train_dataset = torch.utils.data.TensorDataset(Xtrain,Ytrain)
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=1,shuffle=True)

# test_dataset = torch.utils.data.TensorDataset(Xtest,Ytest)
# test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=1,shuffle=True)

In [3]:
wandb.init(project="cell_type_classification",config={})

for e in tqdm(range(epochs)):
    net.eval()
    predTest = net(Xtest)
    predTrain = net(Xtrain)
    lossTest = loss_fn(predTest,Ytest).item()
    loss_fn.zero_grad()
    lossTrain = loss_fn(predTrain,Ytrain).item()
    loss_fn.zero_grad()
    
    wandb.log({"train_loss": lossTrain, "test_loss": lossTest})

    net.train()
    for i, (batch_X, batch_Y) in enumerate(train_dataloader):
        pred = net(batch_X)
        loss = loss_fn(pred,batch_Y)
        loss.backward()
        optim.step()
        optim.zero_grad()

wandb: Currently logged in as: easwaran to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


  0%|          | 0/100 [00:00<?, ?it/s]

: 